In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from transformers import RobertaTokenizer, RobertaConfig
import torch
import csv

In [ ]:
os.chdir('/content/drive/MyDrive/text_VA_prediction/')

In [ ]:
from training import training
from models import RobertaForSequenceClassificationSig

In [ ]:
params = {
  'batch_size' : 32,    # 增大批次大小
  'lr' : 1e-5,          # 增大学习率
  'train_epochs' : 30,  # 增加训练轮数
  'weight_decay' : 0.001, # 减小权重衰减
  'warmup_ratio': 0.05,  # 降低预热比例
}

In [ ]:
class MyDataSetNew(Dataset):
    def __init__(self, filename, maxlen):

        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

        df = pd.read_csv(filename, index_col=0, dtype={'index':np.int32,'text':str,'V':np.float64, 'A':np.float64})

        self.texts = df['text'].to_list()
        self.valence = df['V'].to_list()
        self.arousal = df['A'].to_list()
        self.maxlen = maxlen

    def __getitem__(self, idx):
        item = { }
        aux = self.tokenizer(self.texts[idx], max_length=self.maxlen, truncation=True, padding=False)
        item['input_ids'] = torch.tensor(aux['input_ids'])
        item['attention_mask'] = torch.tensor(aux['attention_mask'])
        item['labels'] = torch.tensor( [ self.valence[idx], self.arousal[idx] ] )

        return item

    def __len__(self):
        return len(self.texts)

In [ ]:
tt = pd.read_csv('/content/drive/MyDrive/text_VA_prediction/corpus/train_data.csv',
index_col=0,
dtype={'index':np.int32,'text':str,'V':np.float64, 'A':np.float64})

In [ ]:
tt['text'][58]

'afk weekend wet weather camping skill'

In [ ]:
def filter_text_column(dataframe, column_name):
    # Keep only entries that are strings
    filtered_df = dataframe[dataframe[column_name].apply(lambda x: isinstance(x, str))]
    return filtered_df

tt = filter_text_column(tt, 'text')

In [ ]:
import pandas as pd

def check_text_column(dataframe, column_name):
    # Check if all entries are strings
    all_strings = dataframe[column_name].apply(lambda x: isinstance(x, str)).all()

    # Check for empty strings
    empty_strings = (dataframe[column_name] == "").any()

    if all_strings and not empty_strings:
        print("All entries are valid strings and none are empty.")
    elif not all_strings:
        non_string_indices = dataframe[~dataframe[column_name].apply(lambda x: isinstance(x, str))].index.tolist()
        print(f"Some entries are not strings at indices: {non_string_indices}")
    if empty_strings:
        empty_indices = dataframe[dataframe[column_name] == ""].index.tolist()
        print(f"There are empty strings in the column at indices: {empty_indices}")


check_text_column(tt, 'text')

All entries are valid strings and none are empty.


In [ ]:
texts = tt['text'].to_list()
valence = tt['V'].to_list()
arousal = tt['A'].to_list()

In [ ]:
print(texts[0:5])

['anyone knows anything history knows great social changes impossible without feminine upheaval social progress measured exactly social position fair sex ugly ones included person', 'nathan spencer offer help', 'press sees hope mecca talks', 'saw lot kids young men dads structure couldnt read beginning realize growing old crack dealer either die go jail', 'holy geez thank much birthday wishes bit overwhelming amazing definitely excellent birthweek many well wishing humbling moments beautiful everything']


In [ ]:
filename_1 = '/content/drive/MyDrive/text_VA_prediction/corpus/train_data.csv'
filename_2 = '/content/drive/MyDrive/text_VA_prediction/corpus/test_data.csv'
train_split = MyDataSetNew(filename=filename_1, maxlen=200)
test_split = MyDataSetNew(filename=filename_2, maxlen=200)
dataset = [train_split, test_split]

In [ ]:
preds_dir = '/content/drive/MyDrive/text_VA_prediction/'

In [ ]:
config = RobertaConfig(
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
)

In [ ]:
model = RobertaForSequenceClassificationSig(config)

In [ ]:
model.to(device)

RobertaForSequenceClassificationSig(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
      

In [ ]:
training(model, params, dataset, preds_dir)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/content/drive/MyDrive/text_VA_prediction/training.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainerMSE.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Starting


Epoch,Training Loss,Validation Loss,Mse Valence,Mae Valence,Pearson Corr Valence,Mse Arousal,Mae Arousal,Pearson Corr Arousal
1,No log,0.020192,0.013492,0.084581,0.091613,0.026892,0.119479,0.020397
2,0.021400,0.021838,0.014287,0.091406,0.138323,0.029390,0.132726,0.059116
3,0.021400,0.019967,0.013201,0.084966,0.164313,0.026733,0.118758,0.066600
4,0.021400,0.021552,0.013807,0.086344,0.254729,0.029296,0.121801,0.098762
5,0.020700,0.020514,0.014428,0.093803,0.156126,0.026601,0.116553,0.084977
6,0.020700,0.024775,0.022434,0.122088,0.331010,0.027117,0.117040,0.098217
7,0.019700,0.019561,0.012551,0.081138,0.303711,0.026570,0.116993,0.056865
8,0.018700,0.020228,0.012515,0.081799,0.318001,0.027940,0.127328,0.160898
9,0.018700,0.019295,0.011803,0.081087,0.361104,0.026787,0.119446,0.171797
10,0.016900,0.020035,0.012485,0.087059,0.408628,0.027585,0.124097,0.206800


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



 

mse_valence : 0.013492089696228504
mae_valence : 0.08458060026168823
pearson_corr_valence : 0.09161323803462637
mse_arousal : 0.02689233422279358
mae_arousal : 0.11947925388813019
pearson_corr_arousal : 0.020397407517912473

 



...


 

mse_valence : 0.011554197408258915
mae_valence : 0.08132441341876984
pearson_corr_valence : 0.48278904316825544
mse_arousal : 0.031738393008708954
mae_arousal : 0.1291808933019638
pearson_corr_arousal : 0.2336696118681315

 



Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



 

mse_valence : 0.011460631154477596
mae_valence : 0.0809115543961525
pearson_corr_valence : 0.48401119783874486
mse_arousal : 0.03197309002280235
mae_arousal : 0.1296696811914444
pearson_corr_arousal : 0.2334028821168222

 



Could not locate the best model at /content/drive/MyDrive/text_VA_prediction/checkpoint-2889/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



 

mse_valence : 0.011460631154477596
mae_valence : 0.0809115543961525
pearson_corr_valence : 0.48401119524241665
mse_arousal : 0.03197309002280235
mae_arousal : 0.1296696811914444
pearson_corr_arousal : 0.23340288532358971

 



In [ ]:
 predictions = pd.read_csv('/content/drive/MyDrive/text_VA_prediction/predictions_fold.csv')

In [ ]:
predictions

,Unnamed: 0,0,1
0,0,0.518497,0.447691
1,1,0.526253,0.390543
2,2,0.503622,0.392994
3,3,0.578620,0.453966
4,4,0.567622,0.474958
...,...,...,...
2562,2562,0.588261,0.465130
2563,2563,0.506692,0.355478
2564,2564,0.509176,0.425577
2565,2565,0.513565,0.411342


from matplotlib import pyplot as plt
predictions['Unnamed: 0'].plot(kind='hist', bins=20, title='Unnamed: 0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
predictions['0'].plot(kind='hist', bins=20, title='0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
predictions['1'].plot(kind='hist', bins=20, title='1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
predictions.plot(kind='scatter', x='Unnamed: 0', y='0', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Unnamed: 0']
  ys = series['1']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = predictions.sort_values('Unnamed: 0', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Unnamed: 0')
_ = plt.ylabel('1')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Unnamed: 0']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Unnamed: 0'}, axis=1)
              .sort_values('Unnamed: 0', ascending=True))
  xs = counted['Unnamed: 0']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = predictions.sort_values('Unnamed: 0', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Unnamed: 0')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
predictions['Unnamed: 0'].plot(kind='line', figsize=(8, 4), title='Unnamed: 0')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
predictions['0'].plot(kind='line', figsize=(8, 4), title='0')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
predictions['1'].plot(kind='line', figsize=(8, 4), title='1')
plt.gca().spines[['top', 'right']].set_visible(False)